# MLB Predictor Project

Group 21, Plotters for Success

Gerardo Skrut, Victor Gikunda, Mathew Huang

In [1]:
import pandas as pd
import seaborn as sn
from datetime import datetime

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data Cleaning

Prior to importing the data, we cleaned and explored the existing data.

# Implementing Neural Networks

To Tune or Neural Network (NN), we are using different numbers. To do so, we will use the gridsearch CV function to process our Data 

In [4]:
import sklearn as sk
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

Firstly we will start with our pitching neural network. We start off with scaling our data and reducing our number of dimensions. From there, we will run it through our MLPClassifier Algorithm from Sci-kit learn. We will determine what hyperparameters work best for our neural network by using the GridSearchCV function to get a cross validation accuracy.

In [ ]:
# Create Pipeline of processes to run through
pline = Pipeline([('scaling', sk.preprocessing.StandardScaler()), ('pca', sk.decomposition.PCA()),
                  ('nnet', MLPClassifier())])

# Defines Parameters to Test
param_grid = {
    'pca__n_components':list(range(5,19)),
    'nnet__hidden_layer_sizes':[30, 40, 50, 60],
    'nnet__activation': ['tanh', 'relu'],
    'nnet__alpha':[0.0001, 0.05]

}
# Grid Search + Scoring
gs = GridSearchCV(pline, param_grid, cv = 5, scoring = 'accuracy')

nested_score = sk.model_selection.cross_val_score(gs,data_x,data_y, cv = 5)

print(nested_score)
print("Accuracy: ", nested_score.mean()*100)

After tuning our hyperparameters, we will now build our final model

In [ ]:
# Extract the Best Parameters
gs.fit(data_x, data_y)
best_params = gs.best_params_
print(best_params)


In [ ]:
# Generate Final Algorithm
layer_size = best_params.get('nnet__hidden_layer_sizes')
activation_func = best_params.get('nnet__activation')
model_alpha = best_params.get('nnet__alpha')

final_model = Pipeline([('scaling', sk.preprocessing.StandardScaler()), 
                        ('nnet', MLPClassifier(activation = activation_func, 
                                               hidden_layer_sizes=layer_size, 
                                               alpha=model_alpha))])
